In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2  # cv2 
import os
import random
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score






Preprocess the image 

1- aplly gaussian blur to remove noise


2- apply median blur to remove salt and pepper noise


3- resize image


4- mormalize pixels values

In [3]:
def preprocess_image_fromPath(image_path, target_size=(64, 64)):
    """
    Preprocess the image by resizing, grayscaling, applying noise reduction, and normalizing.
    Args:
        image_path: Path to the image.
        target_size: Target size for resizing.
    Returns:
        Preprocessed image.
    """
    
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at {image_path}")
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply a Median filter for further noise reduction (optional)
    filtered = cv2.medianBlur(blurred, 3)
    
    # Resize to the target size
    resized = cv2.resize(filtered, target_size)
    
    # Normalize pixel values to [0, 1]
    normalized = resized / 255.0
    
    return normalized

def preprocess_image(image, target_size):
    """
    Preprocess the image by resizing, grayscaling, applying noise reduction, and normalizing.
    Args:
        image: Input image.
        target_size: Target size for resizing.
    Returns:
        Preprocessed image.
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply a Median filter for further noise reduction (optional)
    filtered = cv2.medianBlur(blurred, 3)
    
    # Resize to the target size
    resized = cv2.resize(filtered, target_size)
    
    # Normalize pixel values to [0, 1]
    normalized = resized / 255.0
    
    return normalized


Extract HOG features 


def extract_hog_features(...): This function is defined to compute Histogram of Oriented Gradients (HOG) features from a given grayscale image. HOG features describe the distribution of gradient orientations in an image, useful for object detection and recognition.


image: Input parameter for the preprocessed image, expected to be in grayscale format.


orientations: Number of orientation bins used to compute the gradient histogram. Default is 9, meaning the gradient direction is divided into 9 bins (e.g., 0°–20°, 20°–40°, etc.).


pixels_per_cell: Defines the size of each cell in pixels. The gradient histogram is computed for each cell. Default size is 8x8 pixels.


cells_per_block: Specifies how many cells are grouped into a block for normalization. Default is a 2x2 block of cells.


In [5]:

def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    """
    Extract HOG features from the image.
    Args:
        image: Preprocessed image (grayscale).
        orientations: Number of orientation bins.
        pixels_per_cell: Size of the cell in pixels.
        cells_per_block: Number of cells in each block.
    Returns:
        HOG feature vector.
    """


    hog_features = hog(image, orientations=orientations,
                       pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block,
                       block_norm='L2-Hys', visualize=False)
    '''
hog(image, ...): This function, imported from skimage.feature, computes the HOG features for the input image based on the specified parameters.

orientations=orientations: Specifies the number of bins for the gradient histogram.

pixels_per_cell=pixels_per_cell: Defines the dimensions of the cells for which the gradient histogram is computed.

cells_per_block=cells_per_block: Specifies the size of the block used for local contrast normalization.

block_norm='L2-Hys': Indicates the normalization method to use for the blocks. L2-Hys is a widely used normalization technique that improves robustness to lighting and contrast changes.

visualize=False: Disables the visualization of the HOG image, which is useful for debugging but not required for feature extraction.
'''
    return hog_features


Build list of features and labels for training the model 

In [74]:

# def load_dataset(dataset_path, target_size=(64, 64)):
#     """
#     Load images, preprocess, and extract HOG features.
#     Args:
#         dataset_path: Root directory of the dataset.
#         target_size: Target size for resizing.
#     Returns:
#         Features, labels, and class mapping.
#     """
#     features = []
#     labels = []
#     class_mapping = {}

#     # Generate class mapping dynamically
#     class_names = sorted(os.listdir(dataset_path))  # Sort for consistent label assignment
#     for idx, class_name in enumerate(class_names):
#         class_mapping[class_name] = idx  # Assign an index to each class

#     for class_name, label in class_mapping.items():
#         class_path = os.path.join(dataset_path, class_name)
#         for file_name in os.listdir(class_path):
#             image_path = os.path.join(class_path, file_name)
#             # Preprocess the image
#             preprocessed_image = preprocess_image_fromPath(image_path, target_size)
#             # Extract HOG features
#             feature_vector = extract_hog_features(preprocessed_image)
#             features.append(feature_vector)
#             labels.append(label)

#     return np.array(features), np.array(labels), class_mapping

# # Define dataset path
# dataset_path = r"D:\CUFE\GP\Sleep_detection\archive\mrleyedataset"

# # Load dataset
# X, y, class_mapping = load_dataset(dataset_path)

# # Split into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Print the class mapping
# print("Class Mapping:", class_mapping)


In [7]:
import pickle


# pickle_out = open("class_mapping.pickle","wb")
# pickle.dump(class_mapping, pickle_out)
# pickle_out.close()

# pickle_out = open("X_train.pickle","wb")
# pickle.dump(X_train, pickle_out)
# pickle_out.close()

# pickle_out = open("X_test.pickle","wb")
# pickle.dump(X_test, pickle_out)
# pickle_out.close()

# pickle_out = open("y_train.pickle","wb")
# pickle.dump(y_train, pickle_out)
# pickle_out.close()

# pickle_out = open("y_test.pickle","wb")
# pickle.dump(y_test, pickle_out)
# pickle_out.close()

pickle_in = open("class_mapping.pickle","rb")
class_mapping = pickle.load(pickle_in)

# pickle_in = open("X_train.pickle","rb")
# X_train = pickle.load(pickle_in)

# pickle_in = open("X_test.pickle","rb")
# X_test = pickle.load(pickle_in)

# pickle_in = open("y_train.pickle","rb")
# y_train = pickle.load(pickle_in)

# pickle_in = open("y_test.pickle","rb")
# y_test = pickle.load(pickle_in)



1- Train the model

2- Test the model

3- Evaluate the model

In [76]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# # Train the SVM
# svm_model = SVC(kernel='linear', random_state=42)
# svm_model.fit(X_train, y_train)

# # Evaluate the SVM
# y_pred = svm_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"SVM Accuracy: {accuracy * 100:.2f}%")


In [10]:
# save the model to disk
filename = 'SVM_model.sav'
# pickle.dump(svm_model, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)


Test the model on test images

In [11]:
def predict_image_fromPath(image_path, model, class_mapping, target_size=(64, 64)):
    """
    Predict the class of a new image.
    Args:
        image_path: Path to the image.
        model: Trained SVM model.
        class_mapping: Mapping of class names to labels.
        target_size: Target size for resizing.
    Returns:
        Predicted class name.
    """
    # Preprocess and extract features
    preprocessed_image = preprocess_image_fromPath(image_path, target_size)
    feature_vector = extract_hog_features(preprocessed_image).reshape(1, -1)
    # Predict
    prediction = model.predict(feature_vector)[0]
    # Map label back to class name
    for class_name, label in class_mapping.items():
        if label == prediction:
            return class_name

def predict_image(image, model, class_mapping, target_size=(224, 224)):
    """
    Predict the class of a new image.
    Args:
        image: Input image.
        model: Trained model.
        class_mapping: Dictionary mapping class indices to class names.
        target_size: Target size for resizing.
    Returns:
        Predicted class name.
    """
    # Preprocess and extract features
    preprocessed_image = preprocess_image(image, target_size)
    feature_vector = extract_hog_features(preprocessed_image).reshape(1, -1)
    # Predict
    prediction = model.predict(feature_vector)[0]
    # Map label back to class name
    for class_name, label in class_mapping.items():
        if label == prediction:
            return class_name

Test

In [79]:
# prdicting the image
image_path = r"D:\CUFE\GP\Sleep_detection\archive\open_eye.jpg"
predicted_class = predict_image_fromPath(image_path, loaded_model, class_mapping)
print(f"Predicted Class: {predicted_class}")

Predicted Class: Open-Eyes


In [21]:
img = cv2.imread("archive/man_open.png")
# plt.imshow(img)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
i = 0
for (x, y, w, h) in eyes:
    i += 1
    print(f"Eye {i}: ({x}, {y}, {w}, {h})")
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)  
    
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))




NameError: name 'cv2' is not defined

In [20]:
#CROP THE EYES
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
# eyes_roi = []
for x, y, w, h in eyes:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyess = eye_cascade.detectMultiScale(roi_gray)
    if len(eyess) == 0:
        print("eyes are not detected")
    else:
        for (ex, ey, ew, eh) in eyess:
            eyes_roi = roi_color[ey: ey+eh, ex: ex+ew]

plt.imshow(cv2.cvtColor(eyes_roi, cv2.COLOR_BGR2RGB))

NameError: name 'cv2' is not defined

In [19]:
eyes_roi.shape

NameError: name 'eyes_roi' is not defined

In [83]:
# save the image
cv2.imwrite("archive/test.jpg", eyes_roi)

True

In [84]:
test = predict_image_fromPath("archive/test.jpg", loaded_model, class_mapping)
print(f"Predicted Class: {test}")

Predicted Class: Open-Eyes


In [ ]:
# import cv2
# import time
# import winsound  # For beep sound (Windows)


# # Load Haar cascade classifiers
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     cap = cv2.VideoCapture(1)
# if not cap.isOpened():
#     raise IOError("Cannot open webcam")

# eyes_closed_time = None  # Track when eyes were first closed
# ALERT_THRESHOLD = 3  # Seconds before triggering an alert

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.1, 4)

#     eyes_detected = False  # Default: No eyes detected

#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#         roi_gray = gray[y:y + h, x:x + w]
#         roi_color = frame[y:y + h, x:x + w]

#         eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 4)

#         if len(eyes) > 0:
#             eyes_detected = True  # Eyes detected, reset timer
#             for (ex, ey, ew, eh) in eyes:
#                 cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (255, 0,
#                                                                             0), 2)
#                 #use model to predict the eye
#                 eyes_roi = roi_color[ey: ey+eh, ex: ex+ew]
#                 #USING THE MODEL TO PREDICT THE EYE
#                 test = predict_image(eyes_roi, loaded_model, class_mapping)
#                 print(f"Predicted Class: {test}")
    


#     # Check if eyes are closed or not detected
#     if not eyes_detected:
#         if eyes_closed_time is None:  # Start timing when eyes close/disappear
#             eyes_closed_time = time.time()
#         elif time.time() - eyes_closed_time >= ALERT_THRESHOLD:
#             print("ALERT! Eyes closed/not detected for 3 seconds!")
#             winsound.Beep(2500, 1000)  # Beep sound (2500 Hz, 1 sec)
#             eyes_closed_time = None  # Reset timer after alert
#     else:
#         eyes_closed_time = None  # Reset if eyes open

#     cv2.imshow('Drowsiness Detection', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


ValueError: X has 26244 features, but SVC is expecting 1764 features as input.

: 

In [ ]:
import cv2
import numpy as np
import time
import os
import dlib  # For more robust facial landmark detection

# Create directories if they don't exist
if not os.path.exists("archive"):
    os.makedirs("archive")

# Load the HOG-based SVM model for eye state classification
loaded_model = pickle.load(open("SVM_model.sav", 'rb'))
class_mapping = pickle.load(open("class_mapping.pickle", "rb"))

# Initialize face detector and landmark predictor from dlib for better accuracy
try:
    face_detector = dlib.get_frontal_face_detector()
    # You'll need to download this file: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
    # If not available, we'll fall back to Haar cascades
    landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    using_dlib = True
    print("Using dlib for enhanced face and eye detection")
except:
    print("Dlib not available or shape predictor file missing, falling back to Haar cascades")
    using_dlib = False
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # Load both regular eye detector and eye glasses detector
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    eyeglasses_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')

# Constants for eye detection
EYE_AR_THRESH = 0.2       # Eye aspect ratio threshold for closed eyes
EYE_AR_CONSEC_FRAMES = 3  # Number of consecutive frames the eye must be below threshold

# Eye tracking class with enhanced features
class EyeTracker:
    def __init__(self):
        self.state = "Unknown"
        self.confidence = 0.0
        self.closed_time_start = None
        self.alert_active = False
        self.alert_threshold = 3.0  # Seconds
        self.history = []
        self.history_size = 20
        self.consecutive_closed = 0
        self.has_glasses = False
        self.detection_method = None
        self.last_successful_eye_regions = None
        
    def update(self, eye_state, detection_confidence, detection_method):
        # Store the detection method
        self.detection_method = detection_method
        
        # Add to history for smoothing
        self.history.append(eye_state)
        if len(self.history) > self.history_size:
            self.history.pop(0)
        
        # Count state occurrences
        state_counts = {}
        for state in self.history:
            if state in state_counts:
                state_counts[state] += 1
            else:
                state_counts[state] = 1
        
        # Get most common state
        if self.history:
            most_common = max(state_counts.items(), key=lambda x: x[1])
            smoothed_state = most_common[0]
            self.confidence = most_common[1] / len(self.history)
            
            # Update state if confident enough
            if self.confidence > 0.6:  # 60% confidence threshold
                if self.state != smoothed_state:
                    print(f"State changed: {smoothed_state} (via {detection_method})")
                    self.state = smoothed_state
        
        # Track closed eyes for drowsiness detection
        if self.state == "Close-Eyes":
            if self.closed_time_start is None:
                self.closed_time_start = time.time()
                self.consecutive_closed += 1
            else:
                self.consecutive_closed += 1
        else:
            self.closed_time_start = None
            self.consecutive_closed = 0
            self.alert_active = False

    def check_drowsiness(self):
        if self.closed_time_start is not None:
            elapsed = time.time() - self.closed_time_start
            if elapsed >= self.alert_threshold and not self.alert_active:
                self.alert_active = True
                return True
        return False
    
    def set_glasses_status(self, has_glasses):
        self.has_glasses = has_glasses
    
    def get_closed_duration(self):
        if self.closed_time_start is None:
            return 0
        return time.time() - self.closed_time_start

# Function to extract eye regions using dlib landmarks
def get_eye_regions_dlib(frame, gray, rect):
    shape = landmark_predictor(gray, rect)
    landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])
    
    # Extract eye landmarks
    left_eye = landmarks[36:42]  # Left eye landmarks
    right_eye = landmarks[42:48]  # Right eye landmarks
    
    # Create eye region masks
    left_eye_hull = cv2.convexHull(left_eye)
    right_eye_hull = cv2.convexHull(right_eye)
    
    # Draw eye regions on the frame for visualization
    cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)
    
    # Create mask for the eye regions
    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask, [left_eye_hull], -1, 255, -1)
    cv2.drawContours(mask, [right_eye_hull], -1, 255, -1)
    
    # Apply mask to get only the eye regions
    eyes_gray = cv2.bitwise_and(gray, gray, mask=mask)
    
    # Find eye region bounding boxes
    x_left, y_left, w_left, h_left = cv2.boundingRect(left_eye)
    x_right, y_right, w_right, h_right = cv2.boundingRect(right_eye)
    
    # Extract eye images with some padding
    padding = 5
    left_eye_img = frame[max(0, y_left-padding):min(gray.shape[0], y_left+h_left+padding), 
                          max(0, x_left-padding):min(gray.shape[1], x_left+w_left+padding)]
    right_eye_img = frame[max(0, y_right-padding):min(gray.shape[0], y_right+h_right+padding), 
                           max(0, x_right-padding):min(gray.shape[1], x_right+w_right+padding)]
    
    # Calculate eye aspect ratio to detect eyeglasses
    ear_left = eye_aspect_ratio(left_eye)
    ear_right = eye_aspect_ratio(right_eye)
    avg_ear = (ear_left + ear_right) / 2.0
    
    # Glasses detection based on edge detection
    edges = cv2.Canny(eyes_gray, 30, 100)
    glasses_score = np.sum(edges) / (left_eye_hull.shape[0] + right_eye_hull.shape[0])
    has_glasses = glasses_score > 20  # Threshold for glasses detection
    
    return left_eye_img, right_eye_img, has_glasses, avg_ear

# Function to calculate eye aspect ratio
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the vertical eye landmarks
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    
    # Compute the euclidean distance between the horizontal eye landmarks
    C = np.linalg.norm(eye[0] - eye[3])
    
    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

# Function to enhance eye region for better feature extraction
def enhance_eye_region(eye_img):
    if eye_img is None or eye_img.size == 0:
        return None
        
    # Convert to grayscale if it's a color image
    if len(eye_img.shape) == 3:
        eye_gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
    else:
        eye_gray = eye_img.copy()
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    eye_gray = clahe.apply(eye_gray)
    
    # Bilateral filter for noise reduction while preserving edges
    eye_gray = cv2.bilateralFilter(eye_gray, 9, 75, 75)
    
    return eye_gray

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Create tracker
tracker = EyeTracker()

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Create a copy for display
    display = frame.copy()
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE for better contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced_gray = clahe.apply(gray)
    
    eye_state = "Unknown"
    detection_confidence = 0.0
    detection_method = "None"
    has_glasses = False
    
    # Try to detect faces and eyes using available methods
    if using_dlib:
        # Detect faces using dlib
        faces = face_detector(enhanced_gray)
        
        if len(faces) > 0:
            # Process the largest face
            face = max(faces, key=lambda rect: rect.width() * rect.height())
            
            # Get eye regions using facial landmarks
            left_eye, right_eye, has_glasses, eye_ar = get_eye_regions_dlib(display, enhanced_gray, face)
            
            # Save the eye regions for future reference
            if left_eye is not None and left_eye.size > 0 and right_eye is not None and right_eye.size > 0:
                tracker.last_successful_eye_regions = (left_eye, right_eye)
                
                # Determine eye state based on aspect ratio and model prediction
                if eye_ar < EYE_AR_THRESH:
                    eye_state = "Close-Eyes"
                    detection_confidence = 1.0 - (eye_ar / EYE_AR_THRESH)
                    detection_method = "EAR"
                else:
                    # Try to predict using the model
                    try:
                        # Save and predict with the larger eye
                        if left_eye.size > right_eye.size:
                            eye_img = left_eye
                        else:
                            eye_img = right_eye
                            
                        if eye_img.size > 0:
                            cv2.imwrite("archive/test.jpg", eye_img)
                            eye_state = predict_image_fromPath("archive/test.jpg", loaded_model, class_mapping)
                            detection_confidence = 0.8
                            detection_method = "SVM-Model"
                    except Exception as e:
                        print(f"Prediction error: {e}")
                        # Use aspect ratio as fallback
                        eye_state = "Open-Eyes" if eye_ar >= EYE_AR_THRESH else "Close-Eyes"
                        detection_confidence = 0.7
                        detection_method = "EAR-Fallback"
            
            # Set glasses status
            tracker.set_glasses_status(has_glasses)
            
            # Draw face rectangle
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            cv2.rectangle(display, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
    else:
        # Fallback to Haar cascades
        faces = face_cascade.detectMultiScale(enhanced_gray, 1.1, 5, minSize=(100, 100))
        
        if len(faces) > 0:
            # Process the largest face
            x, y, w, h = max(faces, key=lambda face: face[2] * face[3])
            
            # Draw face rectangle
            cv2.rectangle(display, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Extract face region
            face_gray = enhanced_gray[y:y+h, x:x+w]
            face_color = frame[y:y+h, x:x+w]
            
            # Try to detect eyes with glasses first
            eyes_with_glasses = eyeglasses_cascade.detectMultiScale(
                face_gray, 
                scaleFactor=1.1, 
                minNeighbors=5, 
                minSize=(30, 30)
            )
            
            # If found eyes with glasses detector, use those and mark as having glasses
            if len(eyes_with_glasses) > 0:
                eyes = eyes_with_glasses
                has_glasses = True
                detection_method = "Glasses-Cascade"
            else:
                # Try regular eye detector
                eyes = eye_cascade.detectMultiScale(
                    face_gray, 
                    scaleFactor=1.1, 
                    minNeighbors=5, 
                    minSize=(30, 30)
                )
                has_glasses = False
                detection_method = "Regular-Cascade"
            
            # Process detected eyes
            if len(eyes) > 0:
                # Find the largest eye
                ex, ey, ew, eh = max(eyes, key=lambda eye: eye[2] * eye[3])
                
                # Draw eye rectangle
                cv2.rectangle(face_color, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
                
                # Extract and enhance eye region
                eye_roi = face_color[ey:ey+eh, ex:ex+ew]
                
                if eye_roi.size > 0:
                    # Save last successful detection
                    tracker.last_successful_eye_regions = (eye_roi, None)
                    
                    # Save and predict
                    cv2.imwrite("archive/test.jpg", eye_roi)
                    try:
                        eye_state = predict_image_fromPath("archive/test.jpg", loaded_model, class_mapping)
                        detection_confidence = 0.8
                    except:
                        eye_state = "Unknown"
                        detection_confidence = 0.5
            else:
                # No eyes detected, try to use previously detected eye regions if available
                if tracker.last_successful_eye_regions is not None:
                    eye_state = "Eyes Not Detected"
                    detection_confidence = 0.4
                    detection_method = "Last-Known"
    
    # Update tracker with detected eye state
    tracker.update(eye_state, detection_confidence, detection_method)
    
    # Check for drowsiness
    if tracker.check_drowsiness():
        # Alert for drowsiness
        try:
            import winsound
            winsound.Beep(1000, 200)  # Beep at 1000 Hz for 200 ms
        except:
            pass
    
    # Create info panel
    info_panel = np.zeros((200, frame.shape[1], 3), dtype=np.uint8)
    
    # Add information to the panel
    cv2.putText(info_panel, f"Eye State: {tracker.state}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    cv2.putText(info_panel, f"Detection Method: {tracker.detection_method}", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    cv2.putText(info_panel, f"Confidence: {tracker.confidence:.2f}", (10, 90), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    cv2.putText(info_panel, f"Glasses: {'Yes' if tracker.has_glasses else 'No'}", (10, 120), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    # Show drowsiness warning if necessary
    if tracker.closed_time_start is not None:
        elapsed = tracker.get_closed_duration()
        cv2.putText(info_panel, f"Eyes closed for: {elapsed:.1f}s", (10, 150), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
        
        if elapsed >= tracker.alert_threshold:
            cv2.putText(info_panel, "DROWSINESS ALERT!", (10, 190), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
    
    # Combine display and info panel
    combined = np.vstack((display, info_panel))
    
    # Show the result
    cv2.imshow("Drowsiness Detection", combined)
    
    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()


#new code
cv2.destroyAllWindows()

C:\Users\tarek\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


State changed: Eyes Not Detected (via Regular-Cascade, confidence: 1.00)
State changed: Open-Eyes (via Glasses-Cascade, confidence: 0.67)
State changed: No Face (via None, confidence: 0.67)
State changed: Open-Eyes (via Glasses-Cascade, confidence: 0.67)
State changed: No Face (via None, confidence: 0.67)
State changed: Open-Eyes (via Last-ROI-Fallback, confidence: 0.67)
State changed: Close-Eyes (via Last-ROI-Fallback, confidence: 0.67)
State changed: Open-Eyes (via Glasses-Cascade, confidence: 0.67)
State changed: No Face (via None, confidence: 0.67)
State changed: Open-Eyes (via Glasses-Cascade, confidence: 0.67)
State changed: No Face (via None, confidence: 0.67)
State changed: Close-Eyes (via Last-ROI-Fallback, confidence: 0.67)
State changed: Open-Eyes (via Glasses-Cascade, confidence: 0.67)
State changed: No Face (via None, confidence: 0.67)
State changed: Open-Eyes (via Regular-Cascade, confidence: 0.67)
State changed: Close-Eyes (via Last-ROI-Fallback, confidence: 0.67)
State 